# 0.0 - Elastic Net Linear Regression

Os parâmetros utilizados para treinar o algoritmo **Elastic Net Linear Regression** serão a penalidade de regularização `alpha`, a razão `l1_ratio` (é o grau de mistura que determina o balanceamento entre as penalidades **L1 - Lasso** e **L2 - Ridge**) e o número máximo de iterações, `max_iter`.</br>Os valores padrão (*default*) nesse caso são **alpha=1.0**, **l1_ratio=0.5** e **max_iter=1000**, segundo a documentação oficial disponível em:</br></br>https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

- `l1_ratio`: the *ElasticNet mixing parameter*, with `0 <= l1_ratio <= 1`.</br>For **l1_ratio = 0** the penalty is an **L2** penalty.</br>For **l1_ratio = 1** it is an **L1** penalty. For **0 < l1_ratio < 1**, the penalty is a **combination** of **L1** and **L2**.

# 1.0 - Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model as lm
import matplotlib.pyplot as plt

from itertools import product

from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# 2.0 - Carregando os dados

In [2]:
# Treino
X_train = pd.read_csv('train/X_training.csv')
y_train = pd.read_csv('train/y_training.csv')

# Validação
X_val = pd.read_csv('validation/X_validation.csv')
y_val = pd.read_csv('validation/y_val.csv')

# Teste
X_test = pd.read_csv('test/X_test.csv')
y_test = pd.read_csv('test/y_test.csv')

# 3.0 - Funções

In [3]:
def metrics(y, y_pred, step, algorithm):
    """
    Calcula várias métricas de avaliação de um modelo de regressão e as retorna em um DataFrame.

    Parâmetros:
    -----------
    y : array-like
        Valores da variável resposta.
    
    y_pred : array-like
        Valores preditos pelo modelo.
    
    step : str
        Etapa do processo de modelagem (ex: 'treino', 'validação', 'teste').
    
    algorithm : str
        Nome do algoritmo usado.

    Retorno:
    --------
    df_metrics : pandas.DataFrame
        DataFrame contendo as métricas calculadas:
        - 'Algorithm': Nome do algoritmo.
        - 'Step': Etapa do processo (treino, validação ou teste).
        - 'R2': Coeficiente de determinação.
        - 'MSE': Erro quadrático médio.
        - 'RMSE': Raiz do erro quadrático médio.
        - 'MAE': Erro absoluto médio.
        - 'MAPE': Erro percentual absoluto médio.
    """
    # Calcula as métricas
    r2 = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    mape = mean_absolute_percentage_error(y, y_pred)

    # Cria o DataFrame com as métricas
    df_metrics = pd.DataFrame({   
        'Algorithm': [algorithm],
        'Step': [step],
        'R2': [np.round(r2, 4)],
        'MSE': [np.round(mse, 4)],
        'RMSE': [np.round(rmse, 4)],
        'MAE': [np.round(mae, 4)],
        'MAPE': [np.round(mape, 4)]
    })
    
    return df_metrics

def exporta_excel(nome_algoritmo):
    """
    Exporta os DataFrames de métricas para arquivos Excel em diferentes pastas.

    Args:
        nome_algoritmo (str): Nome do algoritmo utilizado.

    Returns:
        None
    """
    etapas = ['train', 'validation', 'test']
    df_list = [df_metrics_train, df_metrics_val, df_metrics_test]

    for etapa, df in zip(etapas, df_list):
        df.to_excel(f'{etapa}/metrics/metrics_{etapa}_{nome_algoritmo}.xlsx', index=False)

# 4.0 - Ajustando os dados

In [4]:
# transformação do formato DataFrame para Series, essencialmente um array unidimensional
y_train, y_val, y_test = [ df.loc[:, 'song_popularity'] for df in [y_train, y_val, y_test] ]

# 5.0 - Desempenho do modelo

## 5.1 - Dados de treino

In [5]:
# definição do modelo
elastic_net_train = lm.ElasticNet( random_state = 0 )

# treinamento do algoritmo.
elastic_net_train.fit( X_train, y_train )

# Predição sobre os dados de treino
y_pred_train = elastic_net_train.predict( X_train )

# Métricas
df_metrics_train = metrics(y_train, y_pred_train, 'Train', 'Elastic Net Linear Regression')
df_metrics_train

,Algorithm,Step,R2,MSE,RMSE,MAE,MAPE
0,Elastic Net Linear Regression,Train,0.0078,474.2689,21.7777,17.2995,8.7323


## 5.2 - Dados de validação

In [6]:
# definição do modelo
elastic_net_val = lm.ElasticNet( random_state = 0 )

# treinamento do algoritmo. 
elastic_net_val.fit(X_train, y_train)

# Predição sobre os dados de validação
y_pred_val = elastic_net_val.predict(X_val)

# Métricas
df_metrics_val = metrics(y_val, y_pred_val, 'Validation', 'Elastic Net Linear Regression')
df_metrics_val

,Algorithm,Step,R2,MSE,RMSE,MAE,MAPE
0,Elastic Net Linear Regression,Validation,0.0081,473.6356,21.7632,17.2629,8.694


## 5.3 - Ajuste fino dos hiperparâmetros (Etapa de *Fine Tuning*)

In [17]:
from sklearn.preprocessing import StandardScaler

In [7]:
# Definir intervalo de parâmetros
alpha_array = np.arange(0.5, 11, 0.5)
l1_ratio_array = np.arange(0.1, 1.00, 0.1)
max_iter_list = [100, 500, 1000]

# Criar DataFrame para armazenar resultados
metrics_df = pd.DataFrame(columns=['alpha', 'l1_ratio', 'max_iter', 'R2', 'MSE', 'RMSE', 'MAE', 'MAPE'])

# Usar 'itertools.product' para combinar os valores dos hiper-parâmetros
for m, n, o in product(alpha_array, l1_ratio_array, max_iter_list):
    # Definir e treinar o modelo
    model = lm.ElasticNet(alpha=m, l1_ratio=n, max_iter=o, random_state=0)
    model.fit(X_train, y_train)

    # Predição sobre os dados de validação
    y_pred = model.predict(X_val)
    
    # Calcular métricas de performance
    r2_tuning  = r2_score( y_val, y_pred )
    mse_tuning  = mean_squared_error( y_val, y_pred )
    rmse_tuning = root_mean_squared_error( y_val, y_pred )
    mae_tuning  = mean_absolute_error( y_val, y_pred )
    mape_tuning = mean_absolute_percentage_error( y_val, y_pred )
    
    # Armazenar os resultados em uma nova linha como DataFrame
    new_row = pd.DataFrame({
        'alpha': [m],
        'l1_ratio': [n],
        'max_iter': [o],
        'R2': [r2_tuning],
        'MSE': [mse_tuning],
        'RMSE': [rmse_tuning],
        'MAE': [mae_tuning],
        'MAPE': [mape_tuning]
    })

    # Concatenar a nova linha ao DataFrame existente
    metrics_df = pd.concat([metrics_df, new_row], ignore_index=True)

In [8]:
# Valores das métricas de performance para diferentes combinações dos hiper-parâmetros 
metrics_df

,alpha,l1_ratio,max_iter,R2,MSE,RMSE,MAE,MAPE
0,0.5,0.1,100,1.399549e-02,470.828604,21.698585,17.214639,8.686186
1,0.5,0.1,500,1.399549e-02,470.828604,21.698585,17.214639,8.686186
2,0.5,0.1,1000,1.399549e-02,470.828604,21.698585,17.214639,8.686186
3,0.5,0.2,100,1.365089e-02,470.993154,21.702377,17.216063,8.688429
4,0.5,0.2,500,1.365089e-02,470.993154,21.702377,17.216063,8.688429
...,...,...,...,...,...,...,...,...
562,10.5,0.8,500,-7.197077e-07,477.511956,21.852047,17.352836,8.678722
563,10.5,0.8,1000,-7.197077e-07,477.511956,21.852047,17.352836,8.678722
564,10.5,0.9,100,-7.197077e-07,477.511956,21.852047,17.352836,8.678722
565,10.5,0.9,500,-7.197077e-07,477.511956,21.852047,17.352836,8.678722


Em particular, estamos interessados em buscar o conjunto de hiper-parâmetros que minimize a **Raiz do Erro Quadrático Médio (RMSE)**. 

In [9]:
# exibe as 5 primeiras linhas do dataset ordenado do menor para o maior RMSE
metrics_df.sort_values(['RMSE'], ascending=True)[:5]

,alpha,l1_ratio,max_iter,R2,MSE,RMSE,MAE,MAPE
0,0.5,0.1,100,0.013995,470.828604,21.698585,17.214639,8.686186
1,0.5,0.1,500,0.013995,470.828604,21.698585,17.214639,8.686186
2,0.5,0.1,1000,0.013995,470.828604,21.698585,17.214639,8.686186
3,0.5,0.2,100,0.013651,470.993154,21.702377,17.216063,8.688429
4,0.5,0.2,500,0.013651,470.993154,21.702377,17.216063,8.688429


Observe que as três primeiras linhas acima retornam os mesmos valores das métricas de performance. Com efeito, nessa perspectiva, podemos escolher qualquer um dos três conjuntos de hiper-parâmetros para treinar o modelo final. Contudo, é provável que o modelo mais leve seja aquele treinado com `max_iter=100`. Assim:  

Melhores hiper-parâmetros: `alpha = 0.5`, `l1_ratio=0.1` e `max_iter = 100`

In [10]:
best_alpha = 0.5
best_l1_ratio = 0.1
best_max_iter = 100

## 5.4 - Desempenho do modelo para os dados de teste

In [11]:
# definição do modelo
elastic_net_test = lm.ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio, max_iter=best_max_iter, random_state=0)

# Juntar os dados de treino e validação
elastic_net_test.fit( pd.concat( [X_train, X_val] ),
                      pd.concat( [y_train, y_val] ) )

# Predição sobre os dados de teste
y_pred_test = elastic_net_test.predict( X_test )

# Métricas
df_metrics_test = metrics(y_test, y_pred_test, 'Test', 'Elastic Net Linear Regression')
df_metrics_test

,Algorithm,Step,R2,MSE,RMSE,MAE,MAPE
0,Elastic Net Linear Regression,Test,0.0141,480.0462,21.91,17.4165,8.7361


# 6.0 - Gera as planilhas com os resultados

In [14]:
exporta_excel('elastic_net_regression')